In [1]:
import os
import glob
import pandas as pd
from shutil import copyfile
import pathlib

from skimage import io
import matplotlib.pyplot as plt
import numpy as np


In [2]:
# Find the current working directory
os.getcwd()

'C:\\Users\\u0167473\\OneDrive - KU Leuven\\Projects\\Project_NeuronDetection_KlaraGower'

In [3]:
#change current wd
os.chdir('C:\\Users\\u0167473\\OneDrive - KU Leuven\\Projects\\Project_NeuronDetection_KlaraGower')
os.getcwd()

'C:\\Users\\u0167473\\OneDrive - KU Leuven\\Projects\\Project_NeuronDetection_KlaraGower'

In [4]:
imgfiles = []
for file in glob.glob("Data\\objects\\*.tif"):
    imgfiles.append(file)
    
imgfiles_basenames = [os.path.basename(x) for x in glob.glob("Data\\objects\\*.tif")]


In [5]:
# dictionary of lists 
dict = {'fullnames': imgfiles} 
imgfiles = pd.DataFrame(dict)
imgfiles.head()

,fullnames
0,Data\objects\A21893_ah_1_cell_row1010col149lab...
1,Data\objects\A21893_ah_1_cell_row1027col2066la...
2,Data\objects\A21893_ah_1_cell_row1029col315lab...
3,Data\objects\A21893_ah_1_cell_row1036col974lab...
4,Data\objects\A21893_ah_1_cell_row104col559labe...


In [6]:
with pd.option_context('display.max_colwidth', None):
  display(imgfiles.head(1))

,fullnames
0,Data\objects\A21893_ah_1_cell_row1010col149label1120_labelNegative.tif


In [7]:
imgfiles_labels = imgfiles['fullnames'].apply(lambda st: st[st.find("_label")+1:st.find(".tif")])
imgfiles_labels = imgfiles_labels.str.replace("label","",regex=True)
imgfiles_labels.value_counts()

fullnames
Negative       87962
Normal         27323
GVD             2586
NCI_NFT          653
Unsure           433
non_neurons      425
Not_neuron       248
NFT_NCI          218
nan              170
Threads           78
Ambiguous         60
incorrect          1
Name: count, dtype: int64

In [8]:
label_classes = pd.unique(imgfiles_labels)
label_classes

array(['Negative', 'GVD', 'Normal', 'NCI_NFT', 'Unsure', 'non_neurons',
       'nan', 'Threads', 'incorrect', 'Ambiguous', 'NFT_NCI',
       'Not_neuron'], dtype=object)

In [9]:
imgfiles['labels'] = imgfiles_labels
vals_to_replace = {'Negative':'Negative','non_neurons':'Negative','Not_neuron':'Negative','Threads':'Negative',
                  'GVD':'Positive','Normal':'Positive','NCI_NFT':'Positive','Unsure':'Positive',
                  'Ambiguous':'Positive','NFT_NCI':'Positive'}
imgfiles['labels_for_classifier'] = imgfiles['labels'].map(vals_to_replace).fillna(imgfiles['labels'])
                                                                       
imgfiles['basenames'] = imgfiles_basenames                                                                         

In [10]:
imgfiles['labels_for_classifier'].value_counts()

labels_for_classifier
Negative     88713
Positive     31273
nan            170
incorrect        1
Name: count, dtype: int64

In [11]:
imgfiles.to_csv("Data\\objects_annotations.csv",index=False)

In [12]:
class_to_remove = ['nan', 'incorrect']
imgfiles_subset = imgfiles[ ~imgfiles['labels_for_classifier'].isin(class_to_remove)]
imgfiles_subset['labels_for_classifier'].value_counts()

labels_for_classifier
Negative    88713
Positive    31273
Name: count, dtype: int64

In [13]:
test=imgfiles_subset.sample(frac=0.2,random_state=1234)
train=imgfiles_subset.drop(test.index)

In [14]:
train['labels_for_classifier'].value_counts()

labels_for_classifier
Negative    70963
Positive    25026
Name: count, dtype: int64

In [15]:
test['labels_for_classifier'].value_counts()

labels_for_classifier
Negative    17750
Positive     6247
Name: count, dtype: int64

In [16]:
pathlib.Path("classifier_v2\\data\\train").mkdir(parents=True, exist_ok=True)
src_directory = "Data\\objects"
dest_directory = "classifier_v2\\data\\train"

train['basenames'] = train['basenames'].str.replace(".tif","",regex=True)



,fullnames,labels,labels_for_classifier,basenames
0,Data\objects\A21893_ah_1_cell_row1010col149lab...,Negative,Negative,A21893_ah_1_cell_row1010col149label1120_labelN...
1,Data\objects\A21893_ah_1_cell_row1027col2066la...,GVD,Positive,A21893_ah_1_cell_row1027col2066label1132_labelGVD
2,Data\objects\A21893_ah_1_cell_row1029col315lab...,Negative,Negative,A21893_ah_1_cell_row1029col315label1130_labelN...
3,Data\objects\A21893_ah_1_cell_row1036col974lab...,GVD,Positive,A21893_ah_1_cell_row1036col974label991_labelGVD
4,Data\objects\A21893_ah_1_cell_row104col559labe...,GVD,Positive,A21893_ah_1_cell_row104col559label34_labelGVD
...,...,...,...,...
120152,Data\objects\UL9479_14_ah_3_cell_row953col1521...,Negative,Negative,UL9479_14_ah_3_cell_row953col1521label2384_lab...
120153,Data\objects\UL9479_14_ah_3_cell_row956col2040...,Negative,Negative,UL9479_14_ah_3_cell_row956col2040label2392_lab...
120154,Data\objects\UL9479_14_ah_3_cell_row958col1636...,Negative,Negative,UL9479_14_ah_3_cell_row958col1636label2397_lab...
120155,Data\objects\UL9479_14_ah_3_cell_row975col945l...,Normal,Positive,UL9479_14_ah_3_cell_row975col945label2319_labe...


In [19]:
for fileName in train['basenames']:
    src_file = src_directory + "\\" + fileName + ".tif"
    image_as_np_array = io.imread(src_file)
    dest_file = dest_directory + "\\" + fileName + ".png"
    plt.imsave(dest_file, image_as_np_array)

In [20]:
pathlib.Path("classifier_v2\\data\\test").mkdir(parents=True, exist_ok=True)
src_directory = "Data\\objects"
dest_directory = "classifier_v2\\data\\test"

test['basenames'] = test['basenames'].str.replace(".tif","",regex=True)

for fileName in test['basenames']:
    src_file = src_directory + "\\" + fileName + ".tif"
    image_as_np_array = io.imread(src_file)
    dest_file = dest_directory + "\\" + fileName + ".png"
    plt.imsave(dest_file, image_as_np_array)


In [64]:
#labels = pd.unique(imgfiles['labels_for_classifier'])

#for label in labels:
#    try:
#        os.mkdir(os.path.join("Data\\images_for_classifier_v1", label))
#        src_directory = "Data\\objects\\objects"
#        dest_directory = os.path.join("Data\\images_for_classifier_v1", label)
#        temp = imgfiles[imgfiles['labels_for_classifier'] == label]
#        for fileName in temp['basenames']: 
#            src_file = src_directory + "\\" + fileName 
#            dest_file = dest_directory + "\\" + fileName 
#            copyfile(src_file, dest_file)
#    except FileExistsError:
#        pass

In [44]:
#labels = pd.unique(imgfiles['labels_for_classifier'])
#type(imgfiles['labels_for_classifier'])
#temp = imgfiles[imgfiles['labels_for_classifier'] == labels[1]]

#df[df['Frequency'].isin([0.8,1.6])] 

1         Data\objects\objects\A21893_ah_1_cell_row1027c...
3         Data\objects\objects\A21893_ah_1_cell_row1036c...
4         Data\objects\objects\A21893_ah_1_cell_row104co...
6         Data\objects\objects\A21893_ah_1_cell_row106co...
10        Data\objects\objects\A21893_ah_1_cell_row1128c...
                                ...                        
120122    Data\objects\objects\UL9479_14_ah_3_cell_row63...
120137    Data\objects\objects\UL9479_14_ah_3_cell_row81...
120143    Data\objects\objects\UL9479_14_ah_3_cell_row84...
120145    Data\objects\objects\UL9479_14_ah_3_cell_row87...
120155    Data\objects\objects\UL9479_14_ah_3_cell_row97...
Name: fullnames, Length: 31273, dtype: object

In [62]:
#with pd.option_context('display.max_colwidth', None):
  #display(imgfiles.head(1))

,fullnames,labels,labels_for_classifier,basenames
0,Data\objects\objects\A21893_ah_1_cell_row1010col149label1120_labelNegative.tif,Negative,Negative,A21893_ah_1_cell_row1010col149label1120_labelNegative.tif


In [18]:
#with pd.option_context('display.max_colwidth', None):
 # display(train.head(1))


,fullnames,labels,labels_for_classifier,basenames
0,Data\objects\A21893_ah_1_cell_row1010col149label1120_labelNegative.tif,Negative,Negative,A21893_ah_1_cell_row1010col149label1120_labelNegative
